In [1]:
import numpy.random as nr
from prettytable import PrettyTable

In [2]:
class UniformGenerator:
    def __init__(self, a, b):
        if not 0 <= a <= b:
            raise ValueError('Параметры должны удовлетворять условию 0 <= a <= b')
        self._a = a
        self._b = b

    def next(self):
        return nr.uniform(self._a, self._b)

In [3]:
class PoissonGenerator:
    def __init__(self, la):
        self._la = la
    
    def next(self):
        return nr.poisson(self._la)

In [4]:
class ConstGenerator:
    def __init__(self, c):
        self._c = c
        
    def next(self):
        return self._c

In [5]:
class Generator:
    def __init__(self, generator):
        self._generator = generator
        self._receivers = []
        self._generated_requests = 0
        self._next_event_time = 0
        
    @property
    def next_event_time(self):
        return self._next_event_time
    
    @next_event_time.setter
    def next_event_time(self, time):
        self._next_event_time = time
        
    @property
    def generated_requests(self):
        return self._generated_requests
        

    def add_receiver(self, receiver):
        if receiver not in self._receivers:
            self._receivers.append(receiver)

    def remove_receiver(self, receiver):
        try:
            self._receivers.remove(receiver)
        except KeyError:
            pass
    
    def next_time(self):
        return self._generator.next()
    
    def emit_request(self):
        self._generated_requests += 1
        for rec in self._receivers:
            if rec.receive_request():
                return rec
        else:
            return None

In [6]:
class Queue:
    def __init__(self, max_size):
        self._queued = 0
        self._max_queue_size = max_size
        self._queue_size = max_size
        
    @property
    def max_queue_size(self):
        return self._max_queue_size

    @property
    def current_queue_size(self):
        return self._queue_size
    
    @property
    def queued(self):
        return self._queued
    
    def add(self):
        self._queued += 1
        
    def remove(self):
        self._queued -= 1
        
    def increase_size(self):
        self._queue_size += 1

In [7]:
class Processor(Generator):
    def __init__(self, generator, max_queue_size = 0, return_probability = 0.0):
        super().__init__(generator)
        self._generator = generator
        self._processed_requests = 0
        self._return_probability = return_probability
        self._reentered_requests = 0
        self._queue = Queue(max_queue_size)
        
    @property
    def processed_requests(self):
        return self._processed_requests
    
    @property
    def reentered_requests(self):
        return self._reentered_requests
    
    @property
    def queue(self):
        return self._queue
    
    def process(self):
        if self._queue.queued > 0:
            self._processed_requests += 1
            self._queue.remove()
            self.emit_request()
            if nr.random_sample() < self._return_probability:
                self._reentered_requests += 1
                self.receive_request()

    def receive_request(self):
        if self._queue.max_queue_size == 0:
            if self._queue.queued >= self._queue.current_queue_size:
                self._queue.increase_size()
            self._queue.add()
            return True
        elif self._queue.queued < self._queue.current_queue_size:
            self._queue.add()
            return True
        return False

    def next_time_period(self):
        return self._generator.next()

In [8]:
def modelling(max_requests = 300):
    cl_m = 10
    cl_d = 2
    op1_m = 20
    op1_d = 5
    op2_m = 40
    op2_d = 10
    op3_m = 40
    op3_d = 20
    comp1_c = 15
    comp2_c = 30
    
    clients = Generator(UniformGenerator(cl_m - cl_d, cl_m + cl_d))
    op1 = Processor(UniformGenerator(op1_m - op1_d, op1_m + op1_d), 1)
    op2 = Processor(UniformGenerator(op2_m - op2_d, op2_m + op2_d), 1)
    op3 = Processor(UniformGenerator(op3_m - op3_d, op3_m + op3_d), 1)
    comp1 = Processor(ConstGenerator(comp1_c))
    comp2 = Processor(ConstGenerator(comp2_c))
    
    clients.add_receiver(op1)
    clients.add_receiver(op2)
    clients.add_receiver(op3)
    op1.add_receiver(comp1)
    op2.add_receiver(comp1)
    op3.add_receiver(comp2)
    
    nodes = [clients, op1, op2, op3, comp1, comp2]
    for n in nodes:
        n.next_event_time = 0
        
    aborted = 0
    
    clients.next_event_time = clients.next_time()
    op1.next_event_time = op1.next_time()
    
    while clients.generated_requests < max_requests:
        current_time = clients.next_event_time
        for n in nodes:
            if 0 < n.next_event_time < current_time:
                current_time = n.next_event_time
                
        for n in nodes:
            if current_time == n.next_event_time:
                if not isinstance(n, Processor):
                    assigned_processor = clients.emit_request()
                    if assigned_processor is not None:
                        assigned_processor.next_event_time = (current_time +
                                                              assigned_processor.next_time())
                    else:
                        aborted += 1
                    clients.next_event_time = current_time + clients.next_time()
                else:
                    n.process()
                    if n.queue.queued == 0:
                        n.next_event_time = 0
                    else:
                        n.next_event_time = current_time + n.next_time()
                        
    return [aborted, aborted / max_requests]

In [9]:
def print_results(aborted, aborted_p):
    table = PrettyTable()
    table.field_names = ["", "Число отказов", "Вероятность отказа, %"]
    table.add_row(["min", aborted[0], round(aborted_p[0] * 100, 3)])
    table.add_row(["max", aborted[1], round(aborted_p[1] * 100, 3)])
    print(table)

In [14]:
aborted = []
aborted_p = []

for i in range(10):
    res = modelling(300)
    aborted.append(res[0])
    aborted_p.append(res[1])

print_results([min(aborted), max(aborted)], [min(aborted_p), max(aborted_p)])

+-----+---------------+-----------------------+
|     | Число отказов | Вероятность отказа, % |
+-----+---------------+-----------------------+
| min |       59      |         19.667        |
| max |       68      |         22.667        |
+-----+---------------+-----------------------+
